
<h1><center>Trends In Forest Recovery After Stand Replacing Disturbance: A Spectrotemporal Evaluation Of Productivity In Southeastern Pine Forests</center></h1>

<h4><center> Daniel J. Putnam </center></h4>

<center> For partial fulfillment of the reqiurements for the Master of Science degree </center>
<center> College of Natural Resources and Environment </center>
<center> Virginia Polytechnic Institute and State University </center>


## Analysis Preperation

### _Libraries_

In [1]:
import geemap
import ee
import folium
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image
from datetime import datetime
ee.Initialize()

In [2]:
#ee.Authenticate(auth_mode='paste')

### _Imports_

In [3]:
LS5 = ee.ImageCollection("LANDSAT/LT05/C01/T1_SR") # landsat 5
LS7 = ee.ImageCollection("LANDSAT/LE07/C01/T1_SR") # landsat 7
LS8 = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR") # landsat 8
LCMS = ee.ImageCollection("USFS/GTAC/LCMS/v2021-7") # landscape Change Monitoring System
NLCD_col = ee.ImageCollection("USGS/NLCD_RELEASES/2019_REL/NLCD") # national landcover Database
loblolly = ee.FeatureCollection("users/dputnam21/us_eco_l3_NEW")

### _Priliminary set-up_

In [4]:
# Creating sample date range for disturbances
startingD = ee.Date.fromYMD(1994,1,1)
endingD = ee.Date.fromYMD(2011,12,31)

### _Landsat Preprocessing_

In [5]:
# Cloud masking based on the QA band : code taken from landsat example in data catalog in EE
def LScloudMask(image):
  qa = image.select('pixel_qa')
    # removing cloud pixels if confiance is high, cloud shadow, snow
  cloud = qa.bitwiseAnd(1 << 5).And(qa.bitwiseAnd(1 << 7)) \
            .Or(qa.bitwiseAnd(1 << 3)) \
            .Or(qa.bitwiseAnd(1 << 4))
  return image.updateMask(cloud.Not())

# Going to try removing the coverage overlap between LS5 and LS8 to try and fix some issues
#LS5 = LS5.filterDate(start = '1984-01-01',opt_end = ee.Date('2013-04-11'))
#LS8 = LS8.filterDate(start = ee.Date('2013-04-11'))

# Lansat 5/7 & 8 differ in their band labeling, need to select the bands I'm going to use and rename them to
# match each other before merging collections : bands I need [red,green,NIR,SWIR1,SWIR2]    
LS8BandNames = ee.List(['B4','B3','B5','B6','B7','pixel_qa'])
NewBandNames = ee.List(['B3','B2','B4','B5','B7','pixel_qa'])
LS8 = LS8.select(LS8BandNames,NewBandNames)

# Adding a function to calculate and add an NDVI band for a single image
def addNDVI(image):
  ndvi = image.normalizedDifference(['B4', 'B3']).rename('NDVI')
  return image.addBands(ndvi)

# Adding a function to calculate and add an NBR band for a single image.
def addNBR(image):
  nbr = image.normalizedDifference(['B4', 'B7']).rename('NBR')
  return image.addBands(nbr)

# Adding a function to calculate and add an MBI band for a single image.
def addMBI(image):
  MBI = image.expression(
  "MBI = ((b('B5') - b('B7') - b('B4')) / (b('B5') + b('B7') + b('B4'))) + 0.5")
  return image.addBands(MBI)

# adding the cloud mask per generation
LS5 = LS5.map(LScloudMask)
LS7 = LS7.map(LScloudMask)
LS8 = LS8.map(LScloudMask)

# merging the landsat 5 and 7 collections
LS_stack = LS5.merge(LS8)
LS_stack = LS_stack.merge(LS7)

# data reduction on the image stack
LS_stack = LS_stack.filterBounds(loblolly)

# Adding the indices to the filtered combined Landsat collection
LS_stack_wVI = LS_stack.map(addNDVI)
LS_stack_wVI = LS_stack_wVI.map(addNBR)
LS_stack_wVI = LS_stack_wVI.map(addMBI)

---

## Stand Identification Methods

### _Landcover/Landuse Mask_

In [6]:
# New NLCD/LCMS method
# retrieve NLCD for each year
NLCD_2001 = NLCD_col.filter(ee.Filter.eq('system:index', '2001')).first().select("landcover")
NLCD_2004 = NLCD_col.filter(ee.Filter.eq('system:index', '2004')).first().select("landcover")
NLCD_2006 = NLCD_col.filter(ee.Filter.eq('system:index', '2006')).first().select("landcover")
NLCD_2008 = NLCD_col.filter(ee.Filter.eq('system:index', '2008')).first().select("landcover")
NLCD_2011 = NLCD_col.filter(ee.Filter.eq('system:index', '2011')).first().select("landcover")
NLCD_2013 = NLCD_col.filter(ee.Filter.eq('system:index', '2013')).first().select("landcover")
NLCD_2016 = NLCD_col.filter(ee.Filter.eq('system:index', '2016')).first().select("landcover")
NLCD_2019 = NLCD_col.filter(ee.Filter.eq('system:index', '2019')).first().select("landcover")

# combine NLCD to image collection
NLCDlandcover_col = ee.ImageCollection(ee.List([NLCD_2001,NLCD_2004,NLCD_2006,NLCD_2008,NLCD_2011,NLCD_2013,NLCD_2016,NLCD_2019]))

# Function to remap NLCD classes of interest for conditional layer
def remapNLCD(image):
    image = ee.Image(image)
    image = image.updateMask(ee.Image.constant(42).Or(ee.Image.constant(52)))
    image = image.remap(ee.List([42,52]),ee.List([10,1]),defaultValue = None)
    return image

# Layer containing the summed values of pixels across the collection after remapping
NLCDclassSum = NLCDlandcover_col.map(remapNLCD).reduce(ee.Reducer.sum())
NLCDMask = NLCDclassSum.remap(ee.List([62,71,80]),ee.List([1,1,1]), defaultValue = None)

# retrieve LCMS landuse classification
LCMSlanduseCol = LCMS.select("Land_Use")

# A function to select only forest landuse class
def remapLCMS(image):
    image = ee.Image(image)
    onlyForest = image.remap([3],[1], defaultValue = None)
    return onlyForest

LCMSlanduseSum = LCMSlanduseCol.map(remapLCMS).reduce(ee.Reducer.sum())

# # combining the two layers into a landuse / landcover mask
lulcMask = NLCDMask.updateMask(LCMSlanduseSum.gte(36))
lulcMask = lulcMask.clip(loblolly) # clip mask to study boundaries for better loading

### _LCMS Fast change method_

In [7]:
# Using the LCMS Change metric to identify harvest areas in contrast to the max VI method
# Filtering LCMS for the region and timeframe
LCMSchange = LCMS.select('Change_Raw_Probability_Fast_Loss')

def LCMSchangeSelection(image):
    image = ee.Image(image)
    minConfidence = 70
    gtePercent = image.gte(ee.Image.constant(minConfidence))
    gtePercent = gtePercent.updateMask(gtePercent.eq(1))
    gtePercent = gtePercent.set({'year':image.date().get('year')})
    outImage = gtePercent.updateMask(lulcMask).rename('remapped')
    return outImage

# applying the function to the LCMS
FC_stack = LCMSchange.map(LCMSchangeSelection)

### _Connected Pixel (Min stand size) mask_

In [8]:
# A function to apply a connected pixel mask to the input image
def conectPixls(InImage,minArea,maxPixels):
    pixelCount = InImage.connectedPixelCount(maxPixels,False)
    minPixelCount = ee.Image(minArea).divide(ee.Image.pixelArea())
    outImage = InImage.updateMask(pixelCount.gte(minPixelCount))
    return outImage

# a function to be mapped accross an image collection and annually apply the connected pixels mask, also creates an
# additional band to store the year of disturbance for each pixel
def annualConectPixls(image):
    conectPixlsMasked = conectPixls(image,40000,1024) # minimum stand size of 4 ha (represented in m3), maximum of 92 ha (represented in pixel count) (tool limit)
    imgYear = image.get('year')
    imgYearBand = ee.Image.constant(imgYear).uint16().rename('ChangeY')
    imgYearBand = imgYearBand.updateMask(conectPixlsMasked)
    return conectPixlsMasked.addBands(imgYearBand)

FC_final = FC_stack.map(annualConectPixls)

In [9]:
# creating the summary images
FC_final_changeN = FC_final.select('remapped').reduce(ee.Reducer.sum())
FC_final_firstYear = FC_final.select('ChangeY').reduce(ee.Reducer.min())
FC_final_lastYear = FC_final.select('ChangeY').reduce(ee.Reducer.max())

### _Disturbance Year mask_

In [10]:
# Going to help to keep detected disturbances within a given window of time
# first detected disturbance
FC_final_changeN = FC_final_changeN.updateMask(FC_final_firstYear.gte(startingD.get('year')) \
                                               .And(FC_final_firstYear.lte(endingD.get('year')))
                                              )
# last detected disturbance
FC_final_changeN = FC_final_changeN.updateMask(FC_final_lastYear.gte(startingD.get('year')) \
                                               .And(FC_final_lastYear.lte(endingD.get('year')))
                                              )


In [11]:
# Final potential sample pixels
# An image representing pixels that meet all selection criteria
potentialSamples = ee.Image.toUint8(FC_final_changeN.updateMask(FC_final_changeN.eq(1))).rename('remapped_sum')

### _Filter Selection to Only Include Homogenous, Non-Edge Groups of Pixels_

In [12]:
# Edge avoidence
PS_connectedPixelCount = potentialSamples.reduceNeighborhood(ee.Reducer.count(),
                                                             ee.Kernel.circle(2, 'pixels', False, 1),
                                                             'mask',
                                                             True
                                                            )
potentialSamples2 = potentialSamples.updateMask(PS_connectedPixelCount.gte(13))

---

## Automatic Stand Selection Method

### _Creating Sampling Areas Using Ecoregions_

In [13]:
# Function to convert the ecoregion code to an integer value
def convertPropertyToBand(feat):
    feat = ee.Feature(feat)
    prop = feat.get('US_L3CODE')
    propInt = ee.Number.parse(prop).toInt()
    feat = feat.set({'numericL3ecocode':propInt})
    return feat
loblolly = loblolly.map(convertPropertyToBand)

# Need to convert ecoregion feature collection and the property to integer in order for it to be used 
#     as the 'classBand' in the stratifiedSample fucntion
ecoregionImage = ee.Image(loblolly.reduceToImage(['numericL3ecocode'],ee.Reducer.first()))
ecoregionImage = ecoregionImage.cast({'first':'uint8'})
ecoregionImage = ecoregionImage.clipToCollection(loblolly)

# Adding ecoregion code as band to potential sample pixels
potentialSamples2 = potentialSamples2.addBands(ecoregionImage.select('first').rename('numericL3ecocode'))

### _Creating Random Sample Points_

In [14]:
# Going to try just using the export table function to drive
# samplePoints = potentialSamples2.stratifiedSample(numPoints = 50,
#                                                  region = loblolly,
#                                                  classBand = 'numericL3ecocode',
#                                                  scale = 30,
#                                                  seed = 5,
#                                                  dropNulls = True,
#                                                  geometries = True,
#                                                  )

### _Imports/Exports of Created Data_

In [15]:
## today's date
today = str(datetime.now()).split(" ")[0]
today = today.replace("-","_")
today = "_"+today

In [16]:
# Exporting the points created in the above cell to google drive (only way they will finish processing)
# The export process will take about 15 minutes to complete
#geemap.ee_export_vector_to_drive(samplePoints, 'EE_SamplePoints'+today, 'EarthEngine_Exports', file_format='shp', selectors=None)

In [17]:
# # Importing the points created in the above cell
# samplePoints = ee.FeatureCollection('users/dputnam21/stratifiedSamplePoints_03022022')

In [18]:
# Importing points created in arcpro
samplePoints = ee.FeatureCollection('users/dputnam21/samplePoints_05252022')

In [19]:
# # Exporting to google drive the NLCD/LCMS masked LCMS fast change summary layer
# geemap.ee_export_image_to_drive(potentialSamples2, description='PotentialSamples'+today,region = loblolly.geometry(), folder='EarthEngine_Exports', scale=30)

---

### _Creating Point Buffers for Sampling_

In [20]:
# reduction in the number of samplepoints for methodology testing
numSamples = 500
samplePoints2 = samplePoints.filter(ee.Filter.lt('UniqueID',numSamples))

In [21]:
# defining a function to be mapped over point feature collection to create buffers
def makeBuffers(feat):
    inFeat = ee.Feature(feat)
    buff = inFeat.buffer(distance = 60) # 60 meter buffer = 2 pixel radius to align with circlular kernel mask
    return buff

sampleCircles = samplePoints2.map(makeBuffers)

### Displaying images on the map

In [22]:
# LCMS landcover palette
LCMSlcPalette = ['efff6b','ff2ff8','1b9d0c','97ffff','a1a1a1','c2b34a','1B1716']

Map = geemap.Map(basemap="SATELLITE")
#Map.centerObject(loblolly,7)
Map.centerObject(ee.Feature(ee.Geometry.Point([-77.2013,36.8497, ])),13)

# This is the bottom of the layer order

# Map.addLayer(protectedAreas)
# Map.addLayer(GAP)
Map.addLayer(ecoregionImage.select('first'), vis_params = {'palette': LCMSlcPalette, 'min': 45, 'max':75}, name = 'Ecoregion Code Image',shown = False)
Map.addLayer(NLCDMask, vis_params = {'palette': ['2ca25f'],'min':1,'max':1}, name = 'NLCD Mask', shown = False)
Map.addLayer(lulcMask, vis_params = {'palette': ['99d8c9'],'min':1,'max':1}, name = 'LCMS + NLCD landcover mask', shown = False)
Map.addLayer(FC_stack, vis_params = {'palette': ['e34a33'],'min':0,'max':1}, name = 'FC (>70%) raw', shown = False)
Map.addLayer(FC_final.select('remapped'), vis_params = {'palette': ['3182bd'],'min':0,'max':1}, name = 'FC (>70%) Min Area', shown = False)
Map.addLayer(FC_final_changeN,{'palette':['fee0d2','fc9272','de2d26'],'min':1,'max':5},'LCMS Fast Change Count',True)
Map.addLayer(FC_final_firstYear,{'palette':['edf8b1','7fcdbb','2c7fb8'],'min' : 1995, 'max' : 2010},'LCMS Fast Change Year',False)
Map.addLayer(potentialSamples2.select('remapped_sum'),{'palette':['fee0d2','fc9272','de2d26'],'min':0,'max':1}, name = 'Potential Samples 2', shown = True)
Map.addLayer(samplePoints,{'color':'blue'}, name = 'Stratified Random Samples',shown = True)
# Map.addLayer(NLCDclassSum,vis_params = {'palette':['edf8e9','bae4b3','74c476','31a354','006d2c'],'min':0,'max':80})
# Map.addLayer(ee.Feature(ee.Geometry.Point([-82.1455878,33.6141944])),{'color':'blue'})

# This is the top of the layer order

# Adding a legend for exporting images of layers
legend_keys = ['Final Potential Sample Pixels', 'Edge Pixels Removed']
# colorS can be defined using either hex code or RGB (0-255, 0-255, 0-255)
legend_colors = ['de2d26','fee0d2']

Map.add_legend(
    legend_keys=legend_keys, legend_colors=legend_colors, position='bottomright'
)

Map.addLayerControl()
Map

Map(center=[36.8497, -77.2013], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(ch…

---

## Automatic Stand Selection Method

### _Export of stand attributes & Environmental Variables_

In [23]:
# Import of environmental data
elevation_image = ee.Image("USGS/3DEP/10m").select('elevation')
soilTexture_image = ee.Image("OpenLandMap/SOL/SOL_TEXTURE-CLASS_USDA-TT_M/v02")
climate_col = ee.ImageCollection("NASA/NEX-DCP30_ENSEMBLE_STATS")

In [25]:
Map2 = geemap.Map(basemap="SATELLITE")
Map2.centerObject(loblolly,5)

Map2.addLayer(climate_col.first().select('pr_mean'),vis_params = {'palette': ['white','blue'],'min':0,'max':0.00009,'opacity':0.75}, name = 'Precipitation')
Map2.addLayer(loblolly, name = 'Ecoregion Polygons',shown = True)


Map2.addLayerControl()
Map2

Map(center=[33.95843232835059, -85.41690891233773], controls=(WidgetControl(options=['position', 'transparent_…

### _Ecoregion, Location, Topography, Soil Data_

In [ ]:
# Creating topography variables from the elevation layer
topography = ee.Terrain.products(elevation_image)

In [ ]:
# Extracting topography and soil texture variables

# Mean elevation
stands_elevation_FC = topography.select('elevation').reduceRegions(collection = sampleCircles,
                                                                   reducer = ee.Reducer.mean(),
                                                                   scale = 30
                                                                  )
elevationList = stands_elevation_FC.aggregate_array('mean').getInfo()

# Mean slope
stands_slope_FC = topography.select('slope').reduceRegions(collection = sampleCircles,
                                                           reducer = ee.Reducer.mean(),
                                                           scale = 30
                                                          )
slopeList = stands_slope_FC.aggregate_array('mean').getInfo()

# Mean Aspect
stands_slope_FC = topography.select('aspect').reduceRegions(collection = sampleCircles,
                                                           reducer = ee.Reducer.mean(),
                                                           scale = 30
                                                          )
aspectList = stands_slope_FC.aggregate_array('mean').getInfo()

# Mode Soil Texture @ 0cm from surface
stands_soil0_FC = soilTexture_image.select('b0').reduceRegions(collection = sampleCircles,
                                                               reducer = ee.Reducer.mode(),
                                                               scale = 250
                                                              )
soilList0 = stands_soil_FC.aggregate_array('mode').getInfo()

# Mode Soil Texture @ 30cm from surface
stands_soil30_FC = soilTexture_image.select('b30').reduceRegions(collection = sampleCircles,
                                                               reducer = ee.Reducer.mode(),
                                                               scale = 250
                                                              )
soilList30 = stands_soil_FC.aggregate_array('mode').getInfo()

# Mode Soil Texture @ 100cm from surface
stands_soil100_FC = soilTexture_image.select('b100').reduceRegions(collection = sampleCircles,
                                                               reducer = ee.Reducer.mode(),
                                                               scale = 250
                                                              )
soilList100 = stands_soil_FC.aggregate_array('mode').getInfo()

# the points are ordered weird in earth engine, this will need to be used to rearrange them
IDarray = sampleCircles.aggregate_array('UniqueID').getInfo()

In [ ]:
# Extracting the locational and ecotonal information for each stand, assigning all exported topography and soil attributes
#      to a dataframe to be exported

# Timer
start = datetime.now() # figuring out how long this takes to run
print("Extraction initiated :",start)

# list of unique ID's for sample points
stand_nums = range(0,numSamples)

# Creating a dataframe for the stand attributes
standAttributes = pd.DataFrame(index = stand_nums, columns = ['lat','long','ecoR_code','elevation','slope','aspect','soilCode0','soilCode30','soilCode100'])

# Getting the lat and long values from the feature collection
for ID in stand_nums :
    featureDict = samplePoints2.filter(ee.Filter.eq('UniqueID',ID)).first().getInfo()
    standAttributes.iloc[ID,0] = featureDict['geometry']['coordinates'][1]
    standAttributes.iloc[ID,1] = featureDict['geometry']['coordinates'][0]
    standAttributes.iloc[ID,2] = featureDict['properties']['Classified']
    arrayIndex = IDarray.index(ID)
    standAttributes.iloc[ID,3] = elevationList[arrayIndex]
    standAttributes.iloc[ID,4] = slopeList[arrayIndex]
    standAttributes.iloc[ID,5] = aspectList[arrayIndex]
    standAttributes.iloc[ID,6] = soilList0[arrayIndex]
    standAttributes.iloc[ID,7] = soilList30[arrayIndex]
    standAttributes.iloc[ID,8] = soilList100[arrayIndex]
    
end = datetime.now() # effectively ending the timer
duration = end - start
# nicely printing the ellapsed time
timeList = str(duration).split(':')
print('The time elapsed during this execution of this operation was :','\n',
     timeList[0],'Hour(s)','\n',
     timeList[1],'Minute(s)','\n',
      'and',round(float(timeList[2]),ndigits = 0),'Seconds'
     )
    
standAttributes.to_csv(path_or_buf='C:/R_workspace/standAttributes'+today+'.csv', sep=',', na_rep='', float_format=None, header=True, index=True, mode='w')
    
standAttributes

### _Climate Data Extraction_

In [27]:
# function to provide null value of 0.0 whenever there is missing data
def fillNA(point): 
    infeat = ee.Feature(point)
    val = infeat.get('first')
    newVal = ee.List([val, 0.0]).reduce(ee.Reducer.firstNonNull())
    outfeat = infeat.set({'first':newVal})
    return outfeat

In [57]:
# Extracting time series for each climate scenario for each stand
start = datetime.now() # figuring out how long this takes to run
print("Extraction initiated :",start)

singleYlists = []

scenarioValueLists = []

scenarioList = ['rcp26','rcp45','rcp60','rcp85']

for i in range(len(scenarioList)) :
    print(scenarioList[i])
    climateScenario_col = climate_col.filter(ee.Filter.inList('scenario',[scenarioList[i],'historical']))
    
    for year in range(1984,2022): # 2022 not inclusive, collection 1 ends at end of 2021
        print(year,end = '->-')
        filteredColl = climateScenario_col.filter(ee.Filter.calendarRange(year, year, 'year'))
        threeBandImage = filteredColl.reduce(ee.Reducer.mean()) # mean annual value for each variable
        proj = potentialSamples2.select('remapped_sum').projection()
        if len(threeBandImage.bandNames().getInfo()) < 3 :
            singleYlists.append([None]*len(stand_nums))
        else :
        
            # Extracting precipitation values
            pointPrecipValCol = threeBandImage.select('pr_mean_mean').reduceRegions(collection = sampleCircles,
                                                        reducer = ee.Reducer.first(), 
                                                        crs = proj,
                                                        scale = 30 # Same scale as projection and landsat data
                                                        )

            pointPrecipValCol = pointPrecipValCol.set({'year':year})
            pointPrecipValCol = pointPrecipValCol.map(fillNA) 
            PrecipVals = pointPrecipValCol.aggregate_array('first').getInfo()

            # Extracting min temp values
            pointMinTempValCol = threeBandImage.select('tasmin_mean_mean').reduceRegions(collection = sampleCircles,
                                                        reducer = ee.Reducer.first(), 
                                                        crs = proj,
                                                        scale = 30 # Same scale as projection and landsat data
                                                        )

            pointMinTempValCol = pointMinTempValCol.set({'year':year})
            pointMinTempValCol = pointMinTempValCol.map(fillNA) 
            minTempVals = pointMinTempValCol.aggregate_array('first').getInfo()

            # Extracting max temp values
            pointMaxTempValCol = threeBandImage.select('tasmax_mean_mean').reduceRegions(collection = sampleCircles,
                                                        reducer = ee.Reducer.first(), 
                                                        crs = proj,
                                                        scale = 30 # Same scale as projection and landsat data
                                                        )

            pointMaxTempValCol = pointMaxTempValCol.set({'year':year})
            pointMaxTempValCol = pointMaxTempValCol.map(fillNA) 
            maxTempVals = pointMaxTempValCol.aggregate_array('first').getInfo()

            # Creating a list to store all three variable lists for a given year


            singleYlists.append([PrecipVals,minTempVals,maxTempVals])
            
    scenarioValueLists.append(singleYlists)
    singleYlists = []
    print('\n')

#### ending the timer ####
end = datetime.now() 
duration = end - start
# nicely printing the ellapsed time
timeList = str(duration).split(':')
print('The time elapsed during this execution of this operation was :','\n',
     timeList[0],'Hour(s)','\n',
     timeList[1],'Minute(s)','\n',
      'and',round(float(timeList[2]),ndigits = 0),'Seconds'
     )

Extraction initiated : 2022-06-08 11:47:09.229089
rcp26
1984->-1985->-1986->-1987->-1988->-1989->-1990->-1991->-1992->-1993->-1994->-1995->-1996->-1997->-1998->-1999->-2000->-2001->-2002->-2003->-2004->-2005->-2006->-2007->-2008->-2009->-2010->-2011->-2012->-2013->-2014->-2015->-2016->-2017->-2018->-2019->-2020->-2021->-

rcp45
1984->-1985->-1986->-1987->-1988->-1989->-1990->-1991->-1992->-1993->-1994->-1995->-1996->-1997->-1998->-1999->-2000->-2001->-2002->-2003->-2004->-2005->-2006->-2007->-2008->-2009->-2010->-2011->-2012->-2013->-2014->-2015->-2016->-2017->-2018->-2019->-2020->-2021->-

rcp60
1984->-1985->-1986->-1987->-1988->-1989->-1990->-1991->-1992->-1993->-1994->-1995->-1996->-1997->-1998->-1999->-2000->-2001->-2002->-2003->-2004->-2005->-2006->-2007->-2008->-2009->-2010->-2011->-2012->-2013->-2014->-2015->-2016->-2017->-2018->-2019->-2020->-2021->-

rcp85
1984->-1985->-1986->-1987->-1988->-1989->-1990->-1991->-1992->-1993->-1994->-1995->-1996->-1997->-1998->-1999->-2000->-200

In [78]:
# Pulling the values out of the mess created above
yearList = list(range(1984,2022))

# Have to integrate the weirdly organized uniqueID's
standIDs = pointPrecipValCol.aggregate_array('UniqueID').getInfo()

# Creating a seperate dataframe for each climate variable and each climate scenario
rcp26_precipDF = pd.DataFrame(index = standIDs, columns = yearList)
rcp26_minTempDF = pd.DataFrame(index = standIDs, columns = yearList)
rcp26_maxTempDF = pd.DataFrame(index = standIDs, columns = yearList)

rcp45_precipDF = pd.DataFrame(index = standIDs, columns = yearList)
rcp45_minTempDF = pd.DataFrame(index = standIDs, columns = yearList)
rcp45_maxTempDF = pd.DataFrame(index = standIDs, columns = yearList)

rcp60_precipDF = pd.DataFrame(index = standIDs, columns = yearList)
rcp60_minTempDF = pd.DataFrame(index = standIDs, columns = yearList)
rcp60_maxTempDF = pd.DataFrame(index = standIDs, columns = yearList)

rcp85_precipDF = pd.DataFrame(index = standIDs, columns = yearList)
rcp85_minTempDF = pd.DataFrame(index = standIDs, columns = yearList)
rcp85_maxTempDF = pd.DataFrame(index = standIDs, columns = yearList)

# dealing with the mess, sorting these values out into their respective dataframes 
for scenario in scenarioList :
    scenarioIndex = scenarioList.index(scenario)
    singleScenario_allYears = scenarioValueLists[scenarioIndex]
    for year in yearList :
        yearIndex = yearList.index(year)
        singleYear_allVariables = singleScenario_allYears[yearIndex]
        locals()[scenario+'_precipDF'].iloc[:,yearIndex] = singleYear_allVariables[0]
        locals()[scenario+'_minTempDF'].iloc[:,yearIndex] = singleYear_allVariables[1]
        locals()[scenario+'_maxTempDF'].iloc[:,yearIndex] = singleYear_allVariables[2]

In [80]:
# # Exporting the dataframes for each scenario and value
for scenario in scenarioList :
    locals()[scenario+'_precipDF'].to_csv(path_or_buf="C:/R_workspace/"+"precipDF_"+scenario+today+".csv", sep=',', na_rep='', float_format=None, header=True, index=True, mode='w')
    locals()[scenario+'_minTempDF'].to_csv(path_or_buf="C:/R_workspace/"+"minTempDF_"+scenario+today+".csv", sep=',', na_rep='', float_format=None, header=True, index=True, mode='w')
    locals()[scenario+'_maxTempDF'].to_csv(path_or_buf="C:/R_workspace/"+"maxTempDF_"+scenario+today+".csv", sep=',', na_rep='', float_format=None, header=True, index=True, mode='w')

### _New, newer compositing procedure_

In [ ]:
# enter analysis parameters
compositeMonthStart = 2 
compositeMonthEnd = 3 #inclusive
outputIndex = 'NBR'
compositeStat = 'median'

# prep for function
chart_VI = LS_stack_wVI.filter(ee.Filter.calendarRange(compositeMonthStart,compositeMonthEnd,'month'));

years = ee.List.sequence(1984, 2021)

In [ ]:
# Interting a single constant image into the landsat stack to avoid missing data in 1990
zeroImage = ee.Image.constant(0.0).clipToBoundsAndScale(loblolly.geometry(),scale = 30).toFloat()
zeroImage = zeroImage.rename(['NBR'])
zeroImage = zeroImage.set('system:time_start', ee.Date.fromYMD(1990,2,1).millis())
chart_VI = chart_VI.merge(zeroImage)

In [ ]:
# Trying out a new method because the old method is now too slow
start = datetime.now() # figuring out how long this takes to run
print("Extraction initiated :",start)

listOfLists = []

for year in range(1984,2022): # 2022 not inclusive, collection 1 ends at end of 2021
    print(year)
    if year != 1990 :
        filteredColl = chart_VI.filter(ee.Filter.calendarRange(year, year, 'year'))
        singleImage = filteredColl.select(outputIndex).reduce(ee.Reducer.median()) ## CHANGE COMPOSITE STAT HERE ###
        proj = potentialSamples2.select('remapped_sum').projection()
        pointValCol = singleImage.reduceRegions(collection = sampleCircles,
                                                reducer = ee.Reducer.mean(), 
                                                crs = proj,
                                                scale = 30
                                                )
        pointValCol = pointValCol.set({'year':year})
        pointValCol = pointValCol.map(fillNA) 
        NBRvalues = pointValCol.aggregate_array('mean').getInfo()
        listOfLists.append(NBRvalues)
    else :
        NBRvalues = [0.0]*len(stand_nums)
        listOfLists.append(NBRvalues)

#### ending the timer ####
end = datetime.now() 
duration = end - start
# nicely printing the ellapsed time
timeList = str(duration).split(':')
print('The time elapsed during this execution of this operation was :','\n',
     timeList[0],'Hour(s)','\n',
     timeList[1],'Minute(s)','\n',
      'and',round(float(timeList[2]),ndigits = 0),'Seconds'
     )

In [ ]:
# ### this function was in the middle of the above script (between the two pointValCol assignments)
# ###       I don't think I need it anymore (the function is defined in an above cell) but I'm not 100%

#         def fillNA(point): 
#             infeat = ee.Feature(point)
#             NBRval = infeat.get('mean')
#             newVal = ee.List([NBRval, 0.0]).reduce(ee.Reducer.firstNonNull())
#             outfeat = infeat.set({'mean':newVal})
#             return outfeat

In [ ]:
listIndex = 0
for year in range(1984,2022):
    print(year,'\n')
    print('Size', len(listOfLists[listIndex]),'\n')
    print(listOfLists[listIndex],'\n')
    listIndex += 1

In [ ]:
# client side replacement of null value (0.0) with null object
NDVIvals = listOfLists
for i in range(0,len(NDVIvals)):
    for i2 in range(len(NDVIvals[i])) :
        if NDVIvals[i][i2] == 0 :
            NDVIvals[i][i2] = None

In [ ]:
# Creating the dataframe
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

NDVItsDF = pd.DataFrame(index = IDarray, columns = imageYears)
for c in range(0,len(NDVIvals)):
    NDVItsDF.iloc[:,c] = NDVIvals[c]
        
for col in NDVItsDF:
    NDVItsDF[col] = pd.to_numeric(NDVItsDF[col], errors='coerce')
NDVItsDF

In [ ]:
# # Exporting the dataframe before interpolating missing values
NDVItsDF.to_csv(path_or_buf="C:/R_workspace/timeSeriesDF500"+today+"_wNA.csv", sep=',', na_rep='', float_format=None, header=True, index=True, mode='w')

### Creating time series plots

In [ ]:
%matplotlib inline
import scipy as sp
import scipy.signal as scisig

## interpolation of null values
plotDF = NDVItsDF.interpolate(axis = 'columns',method = 'akima')

# setting up for time-series plots
fig, axs = plt.subplots(7, 3, sharex=False, sharey=True, figsize = (16,25))

for i, ax in enumerate(fig.axes):
    ax.plot(plotDF.iloc[i,].transpose(),label = "Interpolated")
    ax.plot(NDVItsDF.iloc[i,].transpose(),label = "Original")
    ax.scatter(imageYears,NDVItsDF.iloc[i,], color = 'orange',s = 20)
    ax.legend(loc='lower right')
    ax.set_title("Stand"+' '+str(IDarray[i])+' '+"Time-Series")
    
fig.subplots_adjust(left=0.1, bottom=0.1, right=None, top=None, wspace=0.3, hspace=0.5)
fig.text(0.5, 0.04, 'Year', ha='center', va='center')
fig.text(0.06, 0.5, (outputIndex+' '+'value'), ha='center', va='center', rotation='vertical')

fig.savefig("C:/R_workspace/timeSeries_interpolation.svg")

In [ ]:
plotDF

plotDF.to_csv(path_or_buf="C:/R_workspace/timeSeriesDF500"+today+".csv", sep=',', na_rep='', float_format=None, header=True, index=True, mode='w')